# OpenAI API Demo


## Call AWS Lambda

### Create a Layer

- `virtualenv env_name_of_choice && source env_name_of_choice/bin/activate`
- Lambda has issues with some native libraries per [this](https://docs.aws.amazon.com/lambda/latest/dg/python-package.html#python-package-native-libraries), so make sure to run this when installing packages:
```bash
pip install --platform manylinux2014_x86_64 --target=package --implementation cp --python-version 3.x --only-binary=:all: --upgrade <package_name> -t ./theEnvFolder/python
```
or use old version of `pydantic`:
```bash
pip install openai==1.10.0 pydantic==1.10.12 -t ./theEnvFolder/python   # there is a bug in later versions of pydantic, 
```

now just zip the environment folder and upload it to AWS Lambda:
```bash
cd theEnvFolder
zip -r layer.zip . 
aws lambda publish-layer-version --layer-name openai_layer --zip-file fileb://layer.zip --compatible-runtimes python3.11 --no-cli-pager
```


### Create Lambda

Create IAM Role that will allow to execute AWS Lambda.

Make sure Lambda Python version matches the version in the Layer above.

Let's use this code:

```python
import json
from openai import OpenAI
import os

def lambda_handler(event, context):
    
    api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    
    if 'body' in event:  # this is needed when calling Lambda via URL, since URL call and boto3 have different events
        event = json.loads(event.get('body'))
        
    role = event.get('key1', 'You are software engineer')
    question = event.get('key2', 'Tell me a joke')
        
    chat = [{"role": "system", "content": role}]

    chat_history = chat.copy()
    chat_history.append({"role":"user", "content": question})

    reply = client.chat.completions.create(
        model="gpt-4",
        messages=chat_history
        )
    
    text = reply.choices[0].message.content
    
    return {
        'statusCode': 200,
        'body': text
    }
```

There are two ways we can call Lamda function and pass the payload directly (so no need for AWS API Gateway):
- using URL
- using boto3 

Let's demo both, URL might be better when using HTML.

We start with very first instruction for the system:

In [190]:
chat = [{"role": "system", "content": 'You are software engineer'}]

Then we can have a first payload:

In [191]:
chat.extend([{"role": "user", "content": "What is the weather like today?"}])

In [192]:
chat

[{'role': 'system', 'content': 'You are software engineer'},
 {'role': 'user', 'content': 'What is the weather like today?'}]

Let's pass this onto a chat:

In [193]:
import json
from openai import OpenAI
import os
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [200]:
reply = client.chat.completions.create(
        model="gpt-4",
        messages=chat
        )
reply_message = reply.choices[0].message
simple_text = {'role': reply_message.role, 'content':reply_message.content}
simple_text


{'role': 'assistant',
 'content': "Sorry, as an AI, I don't have real-time data or access to external databases such as weather information."}

Let's follow up, with extra question:

In [204]:
def question(chat_history, some_question):
        """We take a chat_history, append a question as a user, then get a reply from the assistant, and append that too

        Args:
            chat_history (list): A list of dictionaries, with each dictionary containing a role and content key
            some_question (string): 
        """
        chat_history.append({"role": "user", "content": some_question})
        reply = client.chat.completions.create(
                model="gpt-4",
                messages=chat
                )
        reply_message = reply.choices[0].message
        chat_history.append({'role': reply_message.role, 'content':reply_message.content})



#### URL call

In [24]:
from IPython.display import Markdown
import requests
import json
import boto3
import json


In [144]:

# Your Lambda function's URL
lambda_url = "https://5faukxw75uazcs2zdl4zbvyg2e0lebie.lambda-url.us-west-1.on.aws/"
print("Please wait while GPT4 is done ...")
headers = {'Content-Type': 'application/json'}
response_url = requests.post(headers=headers, url=lambda_url, json=payload)
Markdown(response_url.text)


Please wait while GPT4 is done ...


Sure, here's a simple implementation of Dijkstra's algorithm in Python.

```python
import sys
import heapq

def dijkstra(graph, start_vertex):
    D = {v:float('inf') for v in graph}
    D[start_vertex] = 0

    priority_queue = [(0, start_vertex)]
    while priority_queue:
        (dist, current_vertex) = heapq.heappop(priority_queue)
        for neighbor, neighbor_dist in graph[current_vertex].items():
            old_dist = D[neighbor]
            new_dist = D[current_vertex] + neighbor_dist
            if old_dist < new_dist:
                continue
            else:
                D[neighbor] = new_dist
            heapq.heappush(priority_queue, (D[neighbor], neighbor))
    return D

# Example of a graph
graph = {
    's': {'a': 2, 'b': 1},
    'a': {'s': 3, 'b': 4, 'c':8},
    'b': {'s': 4, 'a': 2, 'd':2},
    'c': {'a': 2, 'd': 7, 't':4},
    'd': {'b': 1, 'c': 11, 't':5},
    't': {'c': 3, 'd': 5}
}

print(dijkstra(graph, 's'))
```
In this script, `priority_queue` is a min-priority queue implemented with a heap data structure, where vertices are prioritized by their currently known shortest distance from `start_vertex`. `D` is a dictionary mapping vertices to their shortest distances from `start_vertex`.

This program prints the shortest path from the 's' vertex to all other vertices in the graph. Note that this is a very simple implementation and might not be the most optimal for larger data sets. You would typically use a proper priority queue or indexed priority queue implementation in real-world cases.

Second method is using boto3:

In [145]:
lambda_client = boto3.client('lambda')
function_name = 'myGPT'

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType='RequestResponse',  # Use 'Event' for asynchronous execution
    Payload=json.dumps(payload)
)

# Read the response
response_payload = json.loads(response['Payload'].read().decode('utf-8'))
Markdown(response_payload['body'])

Sure, here is a Python version of Dijkstra's Algorithm:

```python
from heapq import heappop, heappush

def dijkstra(graph, start):
    D = {x:float('inf') for x in graph}
    D[start] = 0

    queue = [(0, start)]

    while queue:
        _, current_vertex = heappop(queue)
        for neighbor, distance in graph[current_vertex].items():
            old_distance = D[neighbor]
            new_distance = D[current_vertex] + distance

            if new_distance < old_distance:
                heappush(queue, (new_distance, neighbor))
                D[neighbor] = new_distance

    return D

# usage
# graph = {
#     'A': {'B': 1, 'C': 3},
#     'B': {'A': 1, 'C': 2},
#     'C': {'A': 3, 'B': 2}
# }
#
# distances = dijkstra(graph, 'A')
```

Now, explanation for each blocks:
1. import heapq library: which allows you to convert a regular list to a heap
2. Initialise a dictionary `D` that keeps track of the minimum distance from the `start` node to each node in the graph.
3. Initialise a priority queue (heap) `queue` with tuple (0, start). Use distance as priority.
4. While `queue` is not empty. Remove the vertex from the `queue` that has the smallest distance from start.
5. Iterate through the neighbours of current vertex. For each neighbour, check if the shortest distance to start found so far (`old_distance`) is longer than the shortest distance to start through current_vertex (`new_distance`), update the shortest distance to start of neighbour (`D[neighbor]`) and add (new_distance, neighbour) to `queue`.
6. Return list `D` that keeps track of the shortest distance to `start` of each node.

### Making this into a WebPage

In [147]:
from IPython.display import display, HTML, Javascript
import json

html_code = """
<div>
    <textarea id="user_input" rows="2" cols="50" placeholder="Type your message here..."></textarea>
    <button onclick="sendMessage()">Send</button>
</div>
<div id="chat_area" style="height:200px; overflow:auto; border:1px solid #ccc; margin-top:10px; padding:10px;">
    <p><b>ChatBot</b>: Hi, how can I help you today?</p>
</div>
<script>
function sendMessage(){
    var userText = document.getElementById('user_input').value;
    var chatArea = document.getElementById('chat_area');
    
    // Append user's message to chat area
    chatArea.innerHTML += "<p><b>You typed</b>: " + userText + "</p>";
    
    // Clear the user's input field
    document.getElementById('user_input').value = "";

    // Send the message to Python backend
    var kernel = Jupyter.notebook.kernel;
    var command = "process_message('" + userText + "')";
    kernel.execute(command, {iopub: {output: function(response) {
        var res = response.content.text;
        // Append bot's response to chat area
        chatArea.innerHTML += "<p><b>ChatBot</b>: " + res + "</p>";
        // Scroll to the bottom of chat area
        chatArea.scrollTop = chatArea.scrollHeight;
    }}});
}

function process_message(message) {
    // Here you would add your logic to process the message and generate a response
    // For now, we just echo the message
    return "You said: " + message;
}
</script>
"""

display(HTML(html_code))
